In [ ]:
# main file to simulate robocup environment.
import random
import os
import sys
import numpy as np
import pdb
import time
from scipy.io import loadmat,savemat
#import cv2 as cv

from skimage import transform
from skimage.io import imread, imshow
from scipy.spatial.transform import Rotation as Rot
import matplotlib.pyplot as plt

def simImage(img,H):
    tform = transform.ProjectiveTransform(H)
    img2 = transform.warp(img, tform.inverse, mode="constant")

    return img2


def runSimulate(K,dx,thetax,dy,thetay,dz,thetaz):
    T=np.asarray([dx,dy,dz])
    Rx=[[1,0,0],[0,np.cos(thetax),-np.sin(thetax)],[0,np.sin(thetax),np.cos(thetax)]]
    Ry = [[np.cos(thetay),0,np.sin(thetay)],[0,1,0],[-np.sin(thetay),0,np.cos(thetay)]]
    Rz = [[np.cos(thetaz),-np.sin(thetaz),0],[np.sin(thetaz),np.cos(thetaz),0],[0,0,1]]
    R=np.matmul(np.matmul(Rx,Ry),Rz)
    H=np.column_stack((R[:,0:2],T))
    H=np.matmul(K,H)
    return H


def main():
    img = imread(".\\robocup.jpeg")
    row,col,ch=img.shape
    zoom=600
    rangex=np.linspace(-100,-col,30)
    rangey = np.linspace(100, -row, 30)
    rangea = np.concatenate((np.asarray(np.linspace(-3, 3, 20)),np.asarray([4,5,6])))

    i=0
    for el1 in rangey:
        for el2 in rangex:
            for el in rangea:
                K = np.asarray([[zoom, 0, col/2*el], [0, zoom, row/2], [0, 0, 1]], dtype=np.float)
                H = runSimulate(K,el2,-np.pi/2,100,0,el1,-np.pi/2)
                img_out = simImage(img, H)
                thresh=int(img.shape[0]/2)
                #remove reflection
                img_out[:thresh,:,:]=np.zeros(img[:thresh,:,:].shape)
                plt.figure(num=None, figsize=(8, 6), dpi=80)
                imshow(img_out)
                plt.savefig(f'.\\synthetic_images\\{i}', bbox_inches='tight')
                i = i + 1
    return


if __name__=="__main__":
    main()
